<h1><center>BLOSUM</center></h1>


On va donc construire notre matrice Blosum. BlosumX est une matrice de substitution par bloques, elle est 
obtenue à partir d'un alignement de séquences qui ont au maximum X% d'identité entre elles.

In [30]:
import numpy as np
import pandas as pd
import math

In [31]:
seq_input = ["FPTADAGGRS", "FVTADALGRS", "FPTPDAGLRN", "FVTAEAGLRQ", "FPTAEAGGRS"]

On va créer une liste de tous les aa qu'on a dans nos séquences

In [39]:
letters_in_matrix = sorted(''.join(set(''.join(seq_input)))) # liste alphabétique des aa uniques
dict_letter_int = dict(zip(letters_in_matrix, range(len(letters_in_matrix)))) #dico pour avoir l'indice à partir de l'aa
dict_int_letter = {} #dico qu'on va remplir pour avoir l'aa à partir de l'indice
for k, v in dict_letter_int.items():  #pour les couples (aa, indice) dans le dico aa: indice
    dict_int_letter[v] = k  # on remplie le dico indice aa en inversant clef et valeur
print('aa in matrix: ', letters_in_matrix)

aa in matrix:  ['A', 'D', 'E', 'F', 'G', 'L', 'N', 'P', 'Q', 'R', 'S', 'T', 'V']


On crée ensuite un tableau avec nos séquences d'aa

In [40]:
seq_matrix = np.array([list(s) for s in seq_input])
print(seq_matrix)

[['F' 'P' 'T' 'A' 'D' 'A' 'G' 'G' 'R' 'S']
 ['F' 'V' 'T' 'A' 'D' 'A' 'L' 'G' 'R' 'S']
 ['F' 'P' 'T' 'P' 'D' 'A' 'G' 'L' 'R' 'N']
 ['F' 'V' 'T' 'A' 'E' 'A' 'G' 'L' 'R' 'Q']
 ['F' 'P' 'T' 'A' 'E' 'A' 'G' 'G' 'R' 'S']]


On crée une fonction de visualisation afin d'afficher nos matrice avec les headers associés

In [41]:
def show(matrix, header):
    print(pd.DataFrame(matrix, index=header, columns=header))

On crée ensuite la matrice des Cab values, les fréquences de paires

In [42]:
cab_value = np.zeros((len(letters_in_matrix), len(letters_in_matrix)), dtype=np.int) #crée une matrice carrée d'entiers
for column in seq_matrix.T:  # pour chaque colonne de la matrice de sequences        # nuls de taille du nbr d'aa présents
    for i_index, i_letter in enumerate(column): #pour chaque indice i et aa i de la colonne
        subcolumn = column[i_index+1:]  # on crée une sous-colonne avec un décalage de 1 dans l'indice
        for j_letter in subcolumn: # pour chaque aa j dans la sous colonne 
            cab_value[dict_letter_int[j_letter]][dict_letter_int[i_letter]] += 1              
            cab_value[dict_letter_int[i_letter]][dict_letter_int[j_letter]] += 1
            
for i in range(len(cab_value)):  # on met à 0 tout le triangle supérieur droit mais pas la diagonale
    for j in range(i, len(cab_value)):
        if i != j:
            cab_value[i][j] = 0
        else: 
            cab_value[i][j] = cab_value[i][j]/2
show(cab_value, letters_in_matrix)

    A  D  E   F   G  L  N  P  Q   R  S   T  V
A  16  0  0   0   0  0  0  0  0   0  0   0  0
D   0  3  0   0   0  0  0  0  0   0  0   0  0
E   0  6  1   0   0  0  0  0  0   0  0   0  0
F   0  0  0  10   0  0  0  0  0   0  0   0  0
G   0  0  0   0   9  0  0  0  0   0  0   0  0
L   0  0  0   0  10  1  0  0  0   0  0   0  0
N   0  0  0   0   0  0  0  0  0   0  0   0  0
P   4  0  0   0   0  0  0  3  0   0  0   0  0
Q   0  0  0   0   0  0  1  0  0   0  0   0  0
R   0  0  0   0   0  0  0  0  0  10  0   0  0
S   0  0  0   0   0  0  3  0  3   0  3   0  0
T   0  0  0   0   0  0  0  0  0   0  0  10  0
V   0  0  0   0   0  0  0  6  0   0  0   0  1


On construit ensuite la matrice normalisée des fréquences de paire 

In [43]:
T = np.sum(cab_value)  # T est la somme de toutes les valeurs de la matrice des fréquences de paire
qab_value = np.divide(cab_value, T)  # on divise donc chaque valeur de la matrice précédente par T pour la normalisation
show(qab_value, letters_in_matrix)  # on affiche

      A     D     E    F     G     L     N     P     Q    R     S    T     V
A  0.16  0.00  0.00  0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.0  0.00
D  0.00  0.03  0.00  0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.0  0.00
E  0.00  0.06  0.01  0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.0  0.00
F  0.00  0.00  0.00  0.1  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.0  0.00
G  0.00  0.00  0.00  0.0  0.09  0.00  0.00  0.00  0.00  0.0  0.00  0.0  0.00
L  0.00  0.00  0.00  0.0  0.10  0.01  0.00  0.00  0.00  0.0  0.00  0.0  0.00
N  0.00  0.00  0.00  0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.0  0.00
P  0.04  0.00  0.00  0.0  0.00  0.00  0.00  0.03  0.00  0.0  0.00  0.0  0.00
Q  0.00  0.00  0.00  0.0  0.00  0.00  0.01  0.00  0.00  0.0  0.00  0.0  0.00
R  0.00  0.00  0.00  0.0  0.00  0.00  0.00  0.00  0.00  0.1  0.00  0.0  0.00
S  0.00  0.00  0.00  0.0  0.00  0.00  0.03  0.00  0.03  0.0  0.03  0.0  0.00
T  0.00  0.00  0.00  0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.1  0.00

On va calculer les probabilités pour chaque aa puis construire la matrice des probabilités des paires. 

In [44]:
size = seq_matrix.shape[0]*seq_matrix.shape[1] #on calcule la taille de la matrice des sequences = nombre d'aa au total
unique, counts = np.unique(np.ravel(seq_matrix), return_counts=True) #unique renvoie la liste des aa dans unique et
                                                                     #la liste des occurences de chaque aa dans counts
dict_count_value = dict(zip(unique, counts)) #on crée le dico à partir des duplets des listes unique et counts
dict_freq_value = {}
for letter in dict_count_value:
    dict_freq_value[letter] = dict_count_value[letter]/size #dico qui associe un aa et 
                                                                  #P(a)=occurence(aa)/nb total d'aa

eab_value = np.zeros((len(letters_in_matrix), len(letters_in_matrix)), dtype=np.float) #matrice remplie de 0
for i in range(len(eab_value)): 
    for j in range(i, len(eab_value)):
        if i != j:  #si les aa sont différents
            eab_value[j][i] = 2*dict_freq_value[dict_int_letter[i]]*dict_freq_value[dict_int_letter[j]]
        else:   #si c'est le même aa
            eab_value[j][i] = pow(dict_freq_value[dict_int_letter[i]], 2)
show(eab_value, letters_in_matrix)

        A       D       E      F       G       L       N       P       Q  \
A  0.0324  0.0000  0.0000  0.000  0.0000  0.0000  0.0000  0.0000  0.0000   
D  0.0216  0.0036  0.0000  0.000  0.0000  0.0000  0.0000  0.0000  0.0000   
E  0.0144  0.0048  0.0016  0.000  0.0000  0.0000  0.0000  0.0000  0.0000   
F  0.0360  0.0120  0.0080  0.010  0.0000  0.0000  0.0000  0.0000  0.0000   
G  0.0504  0.0168  0.0112  0.028  0.0196  0.0000  0.0000  0.0000  0.0000   
L  0.0216  0.0072  0.0048  0.012  0.0168  0.0036  0.0000  0.0000  0.0000   
N  0.0072  0.0024  0.0016  0.004  0.0056  0.0024  0.0004  0.0000  0.0000   
P  0.0288  0.0096  0.0064  0.016  0.0224  0.0096  0.0032  0.0064  0.0000   
Q  0.0072  0.0024  0.0016  0.004  0.0056  0.0024  0.0008  0.0032  0.0004   
R  0.0360  0.0120  0.0080  0.020  0.0280  0.0120  0.0040  0.0160  0.0040   
S  0.0216  0.0072  0.0048  0.012  0.0168  0.0072  0.0024  0.0096  0.0024   
T  0.0360  0.0120  0.0080  0.020  0.0280  0.0120  0.0040  0.0160  0.0040   
V  0.0144  0

In [45]:
score_value = np.zeros((len(letters_in_matrix), len(letters_in_matrix)), dtype=np.float) #matrice de score remplie de 0
for i in range(len(score_value)):
    for j in range(i, len(score_value)):
        if qab_value[j][i] != 0.000000:  #si la valeur dans la matrice des qab est différente de 0, on calcule le score
            score_value[j][i] = round(2*math.log(qab_value[j][i]/eab_value[j][i], 2))
        else:  #si la valeur dans la matrice des qab est égale à 0, le score =0
            score_value[j][i] = 0.0
score_value = score_value.astype(np.int) #pour l'affichage on convertit en entiers
show(score_value, letters_in_matrix)

   A  D  E  F  G  L  N  P  Q  R  S  T  V
A  5  0  0  0  0  0  0  0  0  0  0  0  0
D  0  6  0  0  0  0  0  0  0  0  0  0  0
E  0  7  5  0  0  0  0  0  0  0  0  0  0
F  0  0  0  7  0  0  0  0  0  0  0  0  0
G  0  0  0  0  4  0  0  0  0  0  0  0  0
L  0  0  0  0  5  3  0  0  0  0  0  0  0
N  0  0  0  0  0  0  0  0  0  0  0  0  0
P  1  0  0  0  0  0  0  4  0  0  0  0  0
Q  0  0  0  0  0  0  7  0  0  0  0  0  0
R  0  0  0  0  0  0  0  0  0  7  0  0  0
S  0  0  0  0  0  0  7  0  7  0  6  0  0
T  0  0  0  0  0  0  0  0  0  0  0  7  0
V  0  0  0  0  0  0  0  6  0  0  0  0  5


Evidemment, cette matrice BLOSUM créée ci-dessus n'a pas de sens biologique puisque les séquences sont courtes, peu nombreuses ce qui implique qu'on ne peut même pas remplir toutes les cases de la matrice.